# PRFs for subject JB collected 20210402, analyzed using popeye

# Setup

In [ ]:
import os, pickle, glob
from bids import BIDSLayout

import numpy as np

import matplotlib.pyplot as plt

import nibabel as nib

import nilearn
from nilearn.input_data import NiftiMasker
from nilearn.image import get_data, math_img, index_img, load_img

import utils

from PIL import Image
import cv2 as cv

In [ ]:
print(' numpy: ', np.__version__, '\n',
      'nibabel: ', nib.__version__, '\n',
      'nilearn: ', nilearn.__version__, '\n')

In [ ]:
sub = "JB"
ses = "20210402"

root_dir = "/Users/smerdis/data/LGN/BIDS/JB_combined"
raw_data_dir = os.path.abspath(f"{root_dir}/sub-{sub}/")
raw_layout = BIDSLayout(raw_data_dir, validate=False, derivatives=False)
derivs_dir = os.path.abspath(f"{root_dir}/derivatives/prf")
freesurfer_dir = os.path.abspath(f"{root_dir}/derivatives/freesurfer")

out_dir = os.path.abspath(f"{derivs_dir}/sub-{sub}")
preproc_layout = BIDSLayout(out_dir, validate=False)

anat_file = sorted([f for f in raw_layout.get(subject=sub, session=ses, suffix='T1w',
            extension=['nii.gz'], return_type='file')])[0]
print(f"anat file: {anat_file}")
raw_bolds_prf = sorted([f for f in raw_layout.get(subject=sub, session=ses, task='prf', suffix='bold',
            extension=['nii.gz'], return_type='file')])
mcf_bolds_prf = sorted([f for f in preproc_layout.get(subject=sub, session=ses, task='prf', suffix='bold',
            extension=['nii.gz'], return_type='file') if 'mcflirt' in f])

prf_mask_file = os.path.abspath(f"{out_dir}/sub-{sub}_ses-{ses}_mask.nii.gz")


In [ ]:
!ls {out_dir}

In [ ]:
prf_ref_vol_noext = f"sub-{sub}_ses-{ses}_refvol"
prf_ref_vol_name = f"{prf_ref_vol_noext}.nii.gz"
prf_ref_vol_path = f"{out_dir}/{prf_ref_vol_name}"
print(prf_ref_vol_path)
!ls {prf_ref_vol_path}

# Preprocessing

## Make reference volume from first dicom

In [ ]:
first_dicom_prf = "/Users/smerdis/Dropbox/data/LGN/SubJb_Prf_Arjun/flat_dicoms/IM-0005-0001.dcm"

In [ ]:
dcm2niix_cmd = f"dcm2niix -v 2 -s y -f {prf_ref_vol_noext} -z y -o {out_dir} {first_dicom_prf}"
print(dcm2niix_cmd)

## Motion correct BOLD runs to this refvol

In [ ]:
for this_epi in raw_bolds_prf:
    epi_stub_mcf = utils.change_bids_description(this_epi, 'desc-mcflirt')
    if not os.path.isdir(os.path.join(out_dir, f"ses-{ses}", "func")):
        os.makedirs(os.path.join(out_dir, f"ses-{ses}", "func"))
    full_outpath = os.path.join(out_dir, f"ses-{ses}", "func", epi_stub_mcf)
    mcflirt_cmd = f"mcflirt -reffile {prf_ref_vol_path} -mats -plots -report -cost mutualinfo -smooth 16 -in {this_epi} -o {full_outpath}"
    print(mcflirt_cmd)
    #os.system(mcflirt_cmd)

## Make brainmask from first run

In [ ]:
!ls {prf_mask_file}

In [ ]:
prf_masker = NiftiMasker(detrend=False, standardize=False, mask_strategy="epi", t_r=2.0)
prf_masker.fit(mcf_bolds_prf[0])
prf_masker.mask_img_.to_filename(prf_mask_file)
prf_mask_img = prf_masker.mask_img_
print(prf_mask_file, prf_mask_img.shape)

## Read in, highpass filter, normalize to psc, and save

In [ ]:
# m = NiftiMasker(mask_img=prf_mask_file, standardize='psc', detrend=True,
#                 high_pass=0.02, t_r=2, mask_strategy='epi', verbose=1)
# m = NiftiMasker(mask_img=prf_mask_file, standardize='psc', t_r=2,
#                 verbose=1) # no highpass
# preproc_desc = 'desc-preproc-nohighpass'
m = NiftiMasker(mask_img=prf_mask_file, standardize=False, t_r=2, high_pass=0.02,
                verbose=1) # highpass, no psc
preproc_desc = 'desc-preproc-highpassnopsc'

In [ ]:
for epi in mcf_bolds_prf:
    preproc_fn = utils.change_bids_description(epi, preproc_desc)
    print(preproc_fn)
    preproc_func_dir = os.path.join(out_dir, f"ses-{ses}", "func")
    if not os.path.isdir(preproc_func_dir):
        os.mkdir(preproc_func_dir)
    full_outpath = os.path.join(preproc_func_dir, preproc_fn)
    preproc_data = m.fit_transform(epi)
    preproc_img = m.inverse_transform(preproc_data)
    print(full_outpath, preproc_img.shape)
    preproc_img.to_filename(full_outpath)

In [ ]:
!ls {preproc_func_dir}

## Average

In [ ]:
preproc_desc

In [ ]:
preproc_bolds_prf = sorted([f for f in preproc_layout.get(subject=sub, session=ses, task='prf', suffix='bold',
            extension=['nii'], return_type='file') if preproc_desc in f])
print('\n'.join(preproc_bolds_prf))

In [ ]:
# have a look at the affine
print(load_img(preproc_bolds_prf[0]).affine)

In [ ]:
prf_input_img = utils.make_timeseries_for_prf(preproc_bolds_prf)

In [ ]:
prf_input_img.shape

In [ ]:
nib.save(prf_input_img, 
         f"{out_dir}/sub-{sub}_ses-{ses}_{preproc_desc}_mean_trim.nii")

## Try a different approach to see if this fixes prfs - just avg the motion-corrected runs, no conversion to % signal change, no highpass

In [ ]:
mcf_bolds_prf

In [ ]:
prf_input_mcflirtonly = utils.make_timeseries_for_prf(mcf_bolds_prf)
nib.save(prf_input_mcflirtonly, 
         f"{out_dir}/sub-{sub}_ses-{ses}_mcflirt_mean_trim.nii")

### Prf inputs from Dev for single session of his JB scans (not 20210402)
 * For comparison purposes - it looks like even one session of Dev's looks better than mine

In [ ]:
dev_func = nib.load("/Users/smerdis/Desktop/JBdata-fromDev/sub-01_1s_prfmean_attfix.nii.gz")

In [ ]:
dev_func_trim = dev_func.get_fdata()[:, :, :, :138]

In [ ]:
nib.save(nib.Nifti1Image(dev_func_trim, dev_func.affine), "/Users/smerdis/Desktop/JBdata-fromDev/sub-01_1s_prfmean_attfix_trim138.nii.gz")

# Make cortical mask

## Convert some freesurfer mgz files to nifti

In [ ]:
ribbon_mgz = f"{freesurfer_dir}/{sub}/mri/ribbon.mgz"
ribbon_nii = f"{out_dir}/ribbon.nii.gz"
!ls {ribbon_mgz}

In [ ]:
mriconv_cmd = f"mri_convert --in_type mgz --out_type nii --out_orientation RAS \
{freesurfer_dir}/{sub}/mri/brain.mgz {out_dir}/brain.nii.gz"
print(mriconv_cmd)
os.system(mriconv_cmd)
mriconv_cmd = f"mri_convert --in_type mgz --out_type nii --out_orientation RAS \
{freesurfer_dir}/{sub}/mri/wm.seg.mgz {out_dir}/wmseg.nii.gz"
print(mriconv_cmd)
os.system(mriconv_cmd)
mriconv_cmd = f"mri_convert --in_type mgz --out_type nii --out_orientation RAS \
{ribbon_mgz} {ribbon_nii}"
print(mriconv_cmd)
os.system(mriconv_cmd)

## Use FLIRT to register func ref vol to anat, save transform

In [ ]:
flirt_cmd = f"flirt -in {prf_ref_vol_path} -ref {out_dir}/brain.nii.gz -wmseg \
{out_dir}/wmseg.nii.gz -dof 6 -omat {out_dir}/func2brain.mat \
-out {out_dir}/mb3refvol2t1"
print(flirt_cmd)
os.system(flirt_cmd)

In [ ]:
# # Register Dev's EPI to the anat and save it
# flirt_cmd = f"flirt -in {out_dir}/5_epi_retino.nii -ref {out_dir}/brain.nii.gz -wmseg \
# {out_dir}/wmseg.nii.gz -dof 6 -omat {out_dir}/func2brain_dev.mat \
# -out {out_dir}/devepi2t1"
# print(flirt_cmd)
# os.system(flirt_cmd)

## Invert the transform and save it

In [ ]:
# For Dev's data
# invxfm_cmd = f"convert_xfm -omat {out_dir}/brain2func_dev.mat -inverse {out_dir}/func2brain_dev.mat"
# print(invxfm_cmd)
# os.system(invxfm_cmd)

In [ ]:
invxfm_cmd = f"convert_xfm -omat {out_dir}/brain2func.mat -inverse {out_dir}/func2brain.mat"
print(invxfm_cmd)
os.system(invxfm_cmd)

## Apply inverse transform to convert freesurfer outputs to functional space

In [ ]:
brain2func_cmd = f"flirt -ref {prf_ref_vol_path} -in {out_dir}/brain.nii.gz -out {out_dir}/brain_funcspace.nii.gz \
-init {out_dir}/brain2func.mat -applyxfm"
print(brain2func_cmd)
os.system(brain2func_cmd)

In [ ]:
# For Dev's data
# brain2func_cmd = f"flirt -ref {out_dir}/5_epi_retino.nii -in {out_dir}/brain.nii.gz -out {out_dir}/brain_funcspace_dev.nii.gz \
# -init {out_dir}/brain2func_dev.mat -applyxfm"
# print(brain2func_cmd)
# os.system(brain2func_cmd)

## Load ribbon and extract cortex only

In [ ]:
def make_func_parc_mask(ribbon_nii, parc_codes, out_fn):
    ribbon_img = load_img(ribbon_nii)
    ribbon_data = ribbon_img.get_fdata()
    cortex_mask = np.zeros_like(ribbon_data)
    print(np.count_nonzero(cortex_mask))
    for c in parc_codes:
        cortex_mask[ribbon_data==c] = 1
    print(np.count_nonzero(cortex_mask))
    cortex_mask_img = nib.Nifti1Image(cortex_mask, ribbon_img.affine)
    nib.save(cortex_mask_img, out_fn)
    cmd = f"flirt -ref {prf_ref_vol_path} -in {out_fn} -out {out_fn}_funcspace.nii.gz -init {out_dir}/brain2func.mat -applyxfm"
    print(cmd)
    os.system(cmd)

In [ ]:
make_func_parc_mask(ribbon_nii, [3, 42], f"{out_dir}/cortex_mask")

In [ ]:
make_func_parc_mask(ribbon_nii, [3], f"{out_dir}/Lcortex_mask")
make_func_parc_mask(ribbon_nii, [42], f"{out_dir}/Rcortex_mask")

In [ ]:
ribbon2func_cmd = f"flirt -ref {prf_ref_vol_path} -in {out_dir}/ribbon.nii.gz -out {out_dir}/ribbon_funcspace.nii.gz \
-init {out_dir}/brain2func.mat -applyxfm"
print(ribbon2func_cmd)
os.system(ribbon2func_cmd)

In [ ]:
print(f"fsleyes {anat_file} {out_dir}/wmseg.nii.gz {out_dir}/ribbon.nii.gz")


# Load Screenshots to make prfs better

In [ ]:
sshots_dir = f"{out_dir}/Screenshots"

In [ ]:
sshots_dir = f"{out_dir}/Screenshots"
imlist = [
    f"{sshots_dir}/Sc_3JBTest_B_001.png",
    f"{sshots_dir}/Sc_3JBTest_B_002.png",
    f"{sshots_dir}/Sc_3JBTest_B_003.png",
    f"{sshots_dir}/Sc_3JBTest_B_004.png",
    f"{sshots_dir}/Sc_3JBTest_B_005.png",
    f"{sshots_dir}/Sc_3JBTest_B_006.png",
    f"{sshots_dir}/Sc_3JBTest_B_007.png",
    f"{sshots_dir}/Blank.png",
    f"{sshots_dir}/Sc_3JBTest_B_008.png",
    f"{sshots_dir}/Sc_3JBTest_B_009.png",
    f"{sshots_dir}/Sc_3JBTest_B_010.png",
    f"{sshots_dir}/Sc_3JBTest_B_011.png",
    f"{sshots_dir}/Sc_3JBTest_B_012.png",
    f"{sshots_dir}/Sc_3JBTest_B_013.png",
    f"{sshots_dir}/Sc_3JBTest_B_014.png",
    f"{sshots_dir}/Blank.png",
    f"{sshots_dir}/Sc_3JBTest_B_015.png",
    f"{sshots_dir}/Sc_3JBTest_B_016.png",
    f"{sshots_dir}/Sc_3JBTest_B_017.png",
    f"{sshots_dir}/Sc_3JBTest_B_018.png",
    f"{sshots_dir}/Sc_3JBTest_B_019.png",
    f"{sshots_dir}/Sc_3JBTest_B_020.png",
    f"{sshots_dir}/Sc_3JBTest_B_021.png",
    f"{sshots_dir}/Blank.png",
    f"{sshots_dir}/Sc_3JBTest_B_022.png",
    f"{sshots_dir}/Sc_3JBTest_B_023.png",
    f"{sshots_dir}/Sc_3JBTest_B_024.png",
    f"{sshots_dir}/Sc_3JBTest_B_025.png",
    f"{sshots_dir}/Sc_3JBTest_B_026.png",
    f"{sshots_dir}/Sc_3JBTest_B_027.png",
    f"{sshots_dir}/Sc_3JBTest_B_028.png",
    f"{sshots_dir}/Blank.png",
    f"{sshots_dir}/Sc_3JBTest_B_029.png",
    f"{sshots_dir}/Sc_3JBTest_B_030.png",
    f"{sshots_dir}/Sc_3JBTest_B_031.png",
    f"{sshots_dir}/Sc_3JBTest_B_032.png",
    f"{sshots_dir}/Sc_3JBTest_B_033.png",
    f"{sshots_dir}/Sc_3JBTest_B_034.png",
    f"{sshots_dir}/Sc_3JBTest_B_035.png",
    f"{sshots_dir}/Blank.png",
    f"{sshots_dir}/Sc_3JBTest_B_036.png",
    f"{sshots_dir}/Sc_3JBTest_B_037.png",
    f"{sshots_dir}/Sc_3JBTest_B_038.png",
    f"{sshots_dir}/Sc_3JBTest_B_039.png",
    f"{sshots_dir}/Sc_3JBTest_B_040.png",
    f"{sshots_dir}/Sc_3JBTest_B_041.png",
    f"{sshots_dir}/Sc_3JBTest_B_042.png",
    f"{sshots_dir}/Blank.png",
    f"{sshots_dir}/Sc_3JBTest_B_043.png",
    f"{sshots_dir}/Sc_3JBTest_B_044.png",
    f"{sshots_dir}/Sc_3JBTest_B_045.png",
    f"{sshots_dir}/Sc_3JBTest_B_046.png",
    f"{sshots_dir}/Sc_3JBTest_B_047.png",
    f"{sshots_dir}/Sc_3JBTest_B_048.png",
    f"{sshots_dir}/Sc_3JBTest_B_049.png",
    f"{sshots_dir}/Blank.png",
    f"{sshots_dir}/Sc_3JBTest_B_050.png",
    f"{sshots_dir}/Sc_3JBTest_B_051.png",
    f"{sshots_dir}/Sc_3JBTest_B_052.png",
    f"{sshots_dir}/Sc_3JBTest_B_053.png",
    f"{sshots_dir}/Sc_3JBTest_B_054.png",
    f"{sshots_dir}/Sc_3JBTest_B_055.png",
    f"{sshots_dir}/Sc_3JBTest_B_056.png",
    f"{sshots_dir}/Blank.png",
    f"{sshots_dir}/Sc_3JBTest_B_001.png",
    f"{sshots_dir}/Sc_3JBTest_B_002.png",
    f"{sshots_dir}/Sc_3JBTest_B_003.png",
    f"{sshots_dir}/Sc_3JBTest_B_004.png",
    f"{sshots_dir}/Sc_3JBTest_B_005.png",
    f"{sshots_dir}/Sc_3JBTest_B_006.png",
    f"{sshots_dir}/Sc_3JBTest_B_007.png",
    f"{sshots_dir}/Blank.png",
    f"{sshots_dir}/Sc_3JBTest_B_008.png",
    f"{sshots_dir}/Sc_3JBTest_B_009.png",
    f"{sshots_dir}/Sc_3JBTest_B_010.png",
    f"{sshots_dir}/Sc_3JBTest_B_011.png",
    f"{sshots_dir}/Sc_3JBTest_B_012.png",
    f"{sshots_dir}/Sc_3JBTest_B_013.png",
    f"{sshots_dir}/Sc_3JBTest_B_014.png",
    f"{sshots_dir}/Blank.png",
    f"{sshots_dir}/Sc_3JBTest_B_015.png",
    f"{sshots_dir}/Sc_3JBTest_B_016.png",
    f"{sshots_dir}/Sc_3JBTest_B_017.png",
    f"{sshots_dir}/Sc_3JBTest_B_018.png",
    f"{sshots_dir}/Sc_3JBTest_B_019.png",
    f"{sshots_dir}/Sc_3JBTest_B_020.png",
    f"{sshots_dir}/Sc_3JBTest_B_021.png",
    f"{sshots_dir}/Blank.png",
    f"{sshots_dir}/Sc_3JBTest_B_022.png",
    f"{sshots_dir}/Sc_3JBTest_B_023.png",
    f"{sshots_dir}/Sc_3JBTest_B_024.png",
    f"{sshots_dir}/Sc_3JBTest_B_025.png",
    f"{sshots_dir}/Sc_3JBTest_B_026.png",
    f"{sshots_dir}/Sc_3JBTest_B_027.png",
    f"{sshots_dir}/Sc_3JBTest_B_028.png",
    f"{sshots_dir}/Blank.png",
    f"{sshots_dir}/Sc_3JBTest_B_029.png",
    f"{sshots_dir}/Sc_3JBTest_B_030.png",
    f"{sshots_dir}/Sc_3JBTest_B_031.png",
    f"{sshots_dir}/Sc_3JBTest_B_032.png",
    f"{sshots_dir}/Sc_3JBTest_B_033.png",
    f"{sshots_dir}/Sc_3JBTest_B_034.png",
    f"{sshots_dir}/Sc_3JBTest_B_035.png",
    f"{sshots_dir}/Blank.png",
    f"{sshots_dir}/Sc_3JBTest_B_036.png",
    f"{sshots_dir}/Sc_3JBTest_B_037.png",
    f"{sshots_dir}/Sc_3JBTest_B_038.png",
    f"{sshots_dir}/Sc_3JBTest_B_039.png",
    f"{sshots_dir}/Sc_3JBTest_B_040.png",
    f"{sshots_dir}/Sc_3JBTest_B_041.png",
    f"{sshots_dir}/Sc_3JBTest_B_042.png",
    f"{sshots_dir}/Blank.png",
    f"{sshots_dir}/Sc_3JBTest_B_043.png",
    f"{sshots_dir}/Sc_3JBTest_B_044.png",
    f"{sshots_dir}/Sc_3JBTest_B_045.png",
    f"{sshots_dir}/Sc_3JBTest_B_046.png",
    f"{sshots_dir}/Sc_3JBTest_B_047.png",
    f"{sshots_dir}/Sc_3JBTest_B_048.png",
    f"{sshots_dir}/Sc_3JBTest_B_049.png",
    f"{sshots_dir}/Blank.png",
    f"{sshots_dir}/Sc_3JBTest_B_050.png",
    f"{sshots_dir}/Sc_3JBTest_B_051.png",
    f"{sshots_dir}/Sc_3JBTest_B_052.png",
    f"{sshots_dir}/Sc_3JBTest_B_053.png",
    f"{sshots_dir}/Sc_3JBTest_B_054.png",
    f"{sshots_dir}/Sc_3JBTest_B_055.png",
    f"{sshots_dir}/Sc_3JBTest_B_056.png",
    f"{sshots_dir}/Blank.png",
]

In [ ]:
len(imlist)

In [ ]:
save_dir = sshots_dir

In [ ]:
for i in range(len(imlist)):
    im = Image.open(imlist[i])
    if 'Blank' in imlist[i]:
        im.save(f"{save_dir}/screen{i+1:03d}.png",'PNG')
    else:
        im_mat = np.array(im)
        im_mat[im_mat[:,:,0] != 128] = 0
        im_mat[im_mat[:,:,1] != 128] = 0
        im_mat[im_mat[:,:,2] != 128] = 0
        #plt.imshow(im_mat)
        im_mat[im_mat[:,:,0] == 128] = -1
        im_mat[im_mat[:,:,1] == 128] = -1
        im_mat[im_mat[:,:,2] == 128] = -1
        #plt.imshow(im_mat)
        img2ar = Image.fromarray(im_mat.astype('uint8'), 'RGB')
        img2ar.save(f"{save_dir}/screen{i+1:03d}.png",'PNG')


#This will make the images but you should go over them individually to make sure that there are no stray pixels..else it will mess you pRF models
# The manual editing is required because of the complex visual stimuli.

In [ ]:
# #edited_sshots_dir = f"{out_dir}/edited-screenshots"
# for i, s in enumerate(sorted(glob.glob(f"{save_dir}/screen*.png"))):
#     print(i, s)
#     im = Image.open(s)
#     im.save(f"{sshots_dir}/screen{i+65:03d}.png")

In [ ]:
img_list = sorted(glob.glob(f"{save_dir}/screen*.png"))
blanks_to_pad = 138-len(img_list)
img_list.extend([img_list[-1]] * blanks_to_pad)

In [ ]:
len(img_list)

In [ ]:
biglist= [np.array([]) for _ in range(len(img_list))]

#This flips the images around to make it like how popeye uses it
    
for i in range(0,len(img_list)):
    im_read = cv.imread(img_list[i])
    im_rgb = cv.cvtColor(im_read, cv.COLOR_BGR2RGB)
    im_gr = cv.cvtColor(im_rgb, cv.COLOR_BGR2GRAY)
    # im_grd = np.divide(im_gr,255)
    print(img_list[i])
    ret,thresh = cv.threshold(im_gr,0,1,cv.THRESH_BINARY_INV)
    biglist[i] = thresh

bigstack = np.dstack((biglist))
pickle.dump(bigstack, open(f"{out_dir}/bigstack-{sub}.pkl", 'wb'))


## Mask manipulation

In [ ]:
print(prf_mask_file)
!ls {prf_mask_file}

In [ ]:
m = nib.load(f"{out_dir}/cortex_mask_funcspace.nii.gz")
#m = nib.load(prf_mask_file)
m_chop = m.get_fdata().copy()
m_chop[:, 40:, :] = 0
chopped_mask_img = nib.Nifti1Image(m_chop, m.affine, m.header)
chopped_mask_path = f"{out_dir}/sub-{sub}_ses-{ses}_desc-cortex-mask-back40_roi.nii.gz"
nib.save(chopped_mask_img, chopped_mask_path)

In [ ]:
print(f"fsleyes {anat_file} {prf_ref_vol_path} {prf_mask_file} {chopped_mask_path}")


## Look at prf results (prfs run on cluster)

In [ ]:
!ls {out_dir}

In [ ]:
!ls {out_dir}/20210511-104720

In [ ]:
# prf_dir = f"{out_dir}/20210407-143953" # Right hemisphere back 40
# prf_dir = f"{out_dir}/20210407-162615" # Bilateral back 40
# prf_dir = f"{out_dir}/20210415-212542" # Bilateral back 40, fixed prfs based on DV feedback 4/15
# prf_dir = f"{out_dir}/20210416-114708" # Bilateral back 40, no psc/highpass
# prf_dir = f"{out_dir}/20210420-225751" # Bilateral back 40, psc no highpass, mask 0.3 - spike is back! remove psc
# prf_dir = f"{out_dir}/20210421-100152" # Bilateral back 40, no psc/highpass, mask 0.3 - no weird spike, more voxels
# prf_dir = f"{out_dir}/20210421-100528" # Rcortex back40 no psc/highpass
# prf_dir = f"{out_dir}/20210421-224614" # bilateral back40, highpass no psc
# prf_dir = f"{out_dir}/20210422-111325" # mcflirt only, mask 0.3, right hemi
# prf_dir = f"{out_dir}/20210428-114114" # mcflirt only, mask 0.3, left hemi
# prf_dir = f"{out_dir}/20210505-135723"
# prf_dir = f"{out_dir}/20210511-104720" # pRFs using Dev's inputs - can't use refvol and xfms
# prf_dir = f"{out_dir}/20210604-141348" # bilateral back 40, mcflirt only, my data
prf_dir = f"{out_dir}/20210604-142202" # whole brain (not just cortex, should include LGN), mcflirt only, my data

In [ ]:
prf_outputs = glob.glob(f"{prf_dir}/sub-{sub}*desc-prf[!mask]*map*")
print("\n".join(prf_outputs))

In [ ]:
files_volspace = [x for x in prf_outputs if ('space-anat' not in x) \
        and (('rsq' in x) or ('rho' in x) or ('theta' in x)) \
        and 'thresh' not in x
]
print('\n'.join(files_volspace))
print(f"fsleyes {anat_file} {prf_ref_vol_path} {' '.join(files_volspace)}")

#### Threshold by rsq

In [ ]:
rsq_file = [f for f in prf_outputs if 'desc-prfrsq' in f and 'thresh' not in f][0]

for output in glob.glob(f"{prf_dir}/sub-{sub}*desc-prf[!mask]*map*"):
    if 'thresh-' not in output and 'space-func' in output:
        print(output)
        _ = utils.threshold(output, rsq_file, .1, prf_dir)

#### Threshold by ecc (to filter out huge eccentricities that are a result of improper fits)

In [ ]:
ecc_file = [f for f in prf_outputs if 'desc-prfrho' in f and 'thresh' not in f][0]
max_ecc = 10
for output in glob.glob(f"{prf_dir}/sub-{sub}*desc-prf[!mask]*map*"):
    _ = utils.threshold(output, ecc_file, max_ecc, prf_dir, how='less')
#     if f"thresh-prfrsq-0.20" not in output and f"thresh-prfrho" not in output:
#         #_ = threshold(output, ecc_file, max_ecc, prf_dir, how='less')
#         #_ = threshold(output, ecc_file, max_ecc, prf_dir, how='less')
#         print(output)

In [ ]:
files = [x for x in glob.glob(f"{prf_dir}/sub-{sub}*desc-prf[!mask]*map*") if ('space-anat' not in x)
                      and (('desc-prfrsq' in x) or ('desc-prfrho' in x) or ('desc-prftheta' in x))
        ] #and 'thresh-prfrsq' in x]
print('\n'.join(files))
print(f"fsleyes {anat_file} {prf_ref_vol_path} {' '.join(files)}")

In [ ]:
for output in files:
    print(output)
    #if 'thresh-prfrsq' in output:
    utils.prf_to_anat(sub, f"{out_dir}/brain.nii.gz", output, f"{out_dir}/func2brain.mat", prf_dir) # my data
    #utils.prf_to_anat(sub, f"{out_dir}/brain.nii.gz", output, f"{out_dir}/func2brain_dev.mat", prf_dir) # dev's data

In [ ]:
files

In [ ]:
utils.freeview_prfs(sub, 'lh', prf_dir)

In [ ]:
utils.freeview_prfs(sub, 'rh', prf_dir)

In [ ]:
utils.freeview_prfs(sub, 'lh', f"{out_dir}/20210428-114114")